# Executing Circuit Asynchronously

MQC3 provides methods for executing a representation asynchronously.  
{py:class}`~.mqc3.client.MQC3Client` and {py:class}`~.mqc3.client.SimulatorClient` offer APIs to achieve this.

In this document, we will explain how to execute quantum circuits in a non-blocking manner.

## Executing circuit representation

First, we create the circuit below.

![circuit](_images/circuit_repr_sample_circuit.svg)

In [ ]:
from math import pi

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops.intrinsic import ControlledZ, Displacement, Measurement, PhaseRotation

circuit = CircuitRepr("circuit")
circuit.Q(0) | PhaseRotation(phi=pi / 4) | Displacement(1, -1)
circuit.Q(0, 1) | ControlledZ(g=1)
circuit.Q(0) | Measurement(theta=0)
circuit.Q(1) | Measurement(theta=pi / 2)

circuit

### Configuring client

In this document, we use {py:class}`~.mqc3.client.MQC3Client`.

In [ ]:
from mqc3.client import MQC3Client

client = MQC3Client()
client.backend = "emulator"
client.n_shots = 100
# client.token = "YOUR_API_TOKEN"

## Submitting circuit

To submit a circuit as a job, use the {py:meth}`~.mqc3.client.MQC3Client.submit` method. This method returns a unique job ID, which you will need to retrieve the job's status and result later.

In [ ]:
job_id = client.submit(circuit)

job_id

## Retrieving Job Status

To monitor the progress of a submitted job, call the {py:meth}`~.mqc3.client.MQC3Client.submit` method with the job's ID.

The method returns a tuple with three elements: the current status string, a detailed status message (which can be empty), and a collection of execution metadata such as versions of the backend and a series `datetime` for different job stages (submission, compilation, etc.).

In [ ]:
from pprint import pprint

current_status, status_detail, execution_details = client.get_job_status(job_id)

print(f"Job Status: {current_status}")
print(f"Details: {status_detail}")
print("Execution Details:")
pprint(execution_details)

## Waiting for Job Completion

Instead of manually polling the job status yourself, you can use the {py:meth}`~.mqc3.client.MQC3Client.wait_for_completion` method. This is a blocking method that continuously polling the job's status and returns when the job reaches a terminal state such as `COMPLETED`, `FAILED`, `TIMEOUT` and `CANCELLED`.  
The return values is same as the return values of {py:meth}`~.mqc3.client.MQC3Client.get_job_status` for the job's final state.  

You can change the polling interval (in seconds) by setting the {py:attr}`~.mqc3.client.MQC3Client.polling_interval` attribute on the client object.

In [ ]:
final_status, status_detail, execution_details = client.wait_for_completion(job_id)

print(f"Job Status: {current_status}")
print(f"Details: {status_detail}")
print("Execution Details:")
pprint(execution_details)

## Retrieving Job Result

After confirming that the job is Completed, you can retrieve its result using the {py:meth}`~.mqc3.client.MQC3Client.get_job_result` method. On success, it returns an instance of the {py:class}`~.mqc3.client.MQC3ClientResult` class, which contains the execution outcome.

This method will raise an error under the following conditions:

- The job's status is not `COMPLETED`.
- The result cannot be retrieved due to an API error or other issues.

Therefore, it is recommended to check the job's status before calling this method.

In [ ]:
result = client.get_job_result(job_id)

pprint(result)

## Cancelling Job

You can attempt to cancel a job that you have submitted using the {py:meth}`~.mqc3.client.MQC3Client.cancel_job` method.

Crucially, a job can only be cancelled if its current status is `QUEUING`. Attempting to cancel a job that is already running, completed, or has failed will result in an error.

In [ ]:
# client.cancel_job(job_id)